<a href="https://colab.research.google.com/github/Anshulnagar00/My_Git_proj/blob/master/ngrog_colab_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deploy Flask through Colab

### This Notebook Illustrates how you can take the advantage of flask-ngrok tunneling api to deploy and host your web app

> **Steps Involved**






> 1. Grab the Random Url and Upload it to a Realtime Database

In the below python script to create a tunnel for Localhost
the main problem of ngrok to deploy our projects is that everytime we run the Localhost server it generates a tunnel url Randomly say http://127a9323.ngrok.io/ as an example so we have to first grab the random URL generated by ngrok and Store it in a variable in order to link this url with a fixed own url.

Here we are going to Store the URL generated in `start_ngrok()` function in an external file only when the function is called first time so we create boolean variable `cv` in the function to store the data in a txt file


In [0]:
import atexit
import json
import os
import platform
import shutil
import subprocess
import tempfile
import time
import zipfile
from pathlib import Path
from threading import Timer

import requests

def get_command():
    system = platform.system()
    if system == "Darwin":
        command = "ngrok"
    elif system == "Windows":
        command = "ngrok.exe"
    elif system == "Linux":
        command = "ngrok"
    else:
        raise Exception("{system} is not supported".format(system=system))
    return command


def run_ngrok(port):
    command = get_command()
    ngrok_path = str(Path(tempfile.gettempdir(), "ngrok"))
    download_ngrok(ngrok_path)
    executable = str(Path(ngrok_path, command))
    os.chmod(executable, 0o777)
    ngrok = subprocess.Popen([executable, 'http', str(port)])
    atexit.register(ngrok.terminate)
    localhost_url = "http://localhost:4040/api/tunnels"  # Url with tunnel details
    time.sleep(1)
    tunnel_url = requests.get(localhost_url).text  # Get the tunnel information
    j = json.loads(tunnel_url)

    tunnel_url = j['tunnels'][0]['public_url']  # Do the parsing of the get
    tunnel_url = tunnel_url.replace("https", "http")
    return tunnel_url


def download_ngrok(ngrok_path):
    if Path(ngrok_path).exists():
        return
    system = platform.system()
    if system == "Darwin":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-darwin-amd64.zip"
    elif system == "Windows":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-windows-amd64.zip"
    elif system == "Linux":
        url = "https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip"
    else:
        raise Exception(f"{system} is not supported")
    download_path = _download_file(url)
    with zipfile.ZipFile(download_path, "r") as zip_ref:
        zip_ref.extractall(ngrok_path)


def download_file(url):
    local_filename = url.split('/')[-1]
    r = requests.get(url, stream=True)
    download_path = str(Path(tempfile.gettempdir(), local_filename))
    with open(download_path, 'wb') as f:
        shutil.copyfileobj(r.raw, f)
    return download_path


def start_ngrok(port):
    cv=True
    ngrok_address = run_ngrok(port)
    data=ngrok_address
    if cv:
      f=open("Logs.txt",'w')
      f.write(data)
      f.close()
      cv=False
    print(f" * Running on {ngrok_address}")
    print(f" * Traffic stats available on http://127.0.0.1:4040")
    



def run_with_ngrok(app):
    """
    The provided Flask app will be securely exposed to the public internet via ngrok when run,
    and the its ngrok address will be printed to stdout
    :param app: a Flask application object
    :return: None
    """
    old_run = app.run

    def new_run(*args, **kwargs):
        port = kwargs.get('port', 5000)
        thread = Timer(1, start_ngrok, args=(port,))
        thread.setDaemon(True)
        thread.start()
        old_run(*args, **kwargs)
    app.run = new_run


> 2. load the log file created in the previous cell and grab the data and send it to your app through `render_template` . In the Index file grab the data and upload it to Firebase database with java Script



In [0]:
from flask import Flask,render_template

app = Flask(__name__,template_folder='templates')
run_with_ngrok(app)  # Start ngrok when app is run
@app.route("/")
def hello():
    f=open('Logs.txt','r')
    data=f.readline()
    return render_template('index.html',urldata=data)


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bd16e57e.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Apr/2020 12:35:21] "GET / HTTP/1.1" 200 -




>3. open the ngrok.io link first whenever localhost server is deployed

